In [2]:
import configparser
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
import boto3
import awswrangler

In [4]:
config = configparser.ConfigParser()
config.read_file(open('credentials.config'))

In [5]:
db = config['POSTGRES']['PG_DB']
user = config['POSTGRES']['PG_UNAME']
passwd = config['POSTGRES']['PG_PASS']
port = config['POSTGRES']['PG_PORT']
host = config['POSTGRES']['PG_HOST']

In [6]:
credentials = "postgresql://{}:{}@{}:{}/{}".format(user,passwd,host,port,db)

#using psycopg2 to test connection since there are no tables
import psycopg2
try:
    conn = psycopg2.connect(host=host,dbname=db,user=user,password=passwd,port=port)
except Exception as e:
    print(e)
    
conn.set_session(autocommit=True)

try:
    cur = conn.cursor()
    
except:
    print(e)

In [7]:
#Helper functions to work with the database
def schemaGen(dataframe, schemaName):
    localSchema = pd.io.sql.get_schema(dataframe,schemaName)
    localSchema = localSchema.replace('TEXT','VARCHAR(255)').replace('INTEGER','NUMERIC').replace('\n','').replace('"',"")
    return "".join(localSchema)

#Using pandas read_sql for getting schema
def getSchema(tableName, credentials):
    schema = pd.read_sql("""SELECT * FROM information_schema.columns where table_name='{}'""".format(tableName),con=credentials)
    return schema

#Issue is in using pd.read_sql to write data to the database. so using psycopg2
def queryTable(query):
    try:
        schema = cur.execute(query)
        return 
    except Exception as e:
        print(e)
        
#This doesn't return anything

#Using the pd.read_sql for getting data from db
def queryBase(query):
    requiredTable = pd.read_sql(query,con=credentials)
    return requiredTable

#This returns the dataframe

In [10]:
queryBase("""SELECT * FROM test_table""")

,identity,full_name
0,1.0,new year
1,2.0,new moon
2,3.0,new dress


In [ ]:
queryTable("""INSERT INTO janorders(order_id, order_date, order_customer_id, order_status)
              SELECT order_id, order_date, order_customer_id, order_status
              FROM orders
              WHERE to_char(order_date,'yyyy-MM')='2014-01'""")

In [ ]:
queryBase("""SELECT * FROM janorders LIMIT 5""")

In [ ]:
#create the pandas dataframe
!cat source_folder/top_earners_list.csv

In [ ]:
top_earners = pd.read_csv('source_folder/top_earners_list.csv')
top_earners.head()

In [ ]:
top_earners.to_sql()